In [1]:
import torch
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow import keras
import transformers
import numpy as np
from os import path
import pandas as pd
from constants import *
import dataset_utils as du
from new_model import create_model, create_model_no_stackx

In [2]:
# Hyperparameters
TRAIN_RATIO = .6
DEV_RATIO = .2
LEARNING_RATE = 0.000001 
BATCH_SIZE = 3
EPOCHS = 2

In [3]:
# Uncomment the following line to rerun pre-processing and tokenization
# prepare_and_save(RAW_DATASET_PATH, path.join(PROCESSED_DIR, PROCESSED_DATASET))

data = pd.read_pickle(path.join(PROCESSED_DIR, PROCESSED_DATASET))
interest = du.get_interest_dataset(data)
interest = du.get_interest_dataset(data)
interest = du.get_balanced_dataset(interest, 'interest_binary', random_state=1)
interest = interest.sample(frac=1, random_state=1)


/home/ubuntu/Question-Classifier/dataset_utils.py:143: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  interest['interest_binary'] = (interest['interest'] > 3).astype(int)
/home/ubuntu/Question-Classifier/dataset_utils.py:143: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  interest['interest_binary'] = (interest['interest'] > 3).astype(int)


In [4]:
# K-fold split
features = du.get_features(interest)
labels = du.get_labels(interest, 'interest_binary')
folds = du.kfold_split(features, labels, num_folds=8)

# Alternate to k-fold split: manually split

# train_set, dev_set, test_set = du.split_datasets(interest, .2, DEV_RATIO)
# train_features, train_labels = du.get_features(train_set), du.get_labels(train_set, 'interest_binary')
# dev_features, dev_labels = du.get_features(dev_set), du.get_labels(dev_set, 'interest_binary')
# test_features, test_labels = du.get_features(test_set), du.get_labels(test_set, 'interest_binary')

In [7]:
for fold in folds:
    print(du.get_balance(fold['train_labels']))
    
train_features = folds[0]['train_features']
train_labels = folds[0]['train_labels']

{0: 353, 1: 343, 'percent_pos': 0.492816091954023}
{0: 347, 1: 349, 'percent_pos': 0.5014367816091954}
{0: 344, 1: 352, 'percent_pos': 0.5057471264367817}
{0: 352, 1: 344, 'percent_pos': 0.4942528735632184}
{0: 341, 1: 356, 'percent_pos': 0.5107604017216643}
{0: 346, 1: 351, 'percent_pos': 0.503586800573888}
{0: 356, 1: 341, 'percent_pos': 0.48923959827833574}
{0: 347, 1: 350, 'percent_pos': 0.5021520803443329}


In [10]:
keras.backend.clear_session()
model = create_model_no_stackx(MAX_LEN, NUM_SX_FEATURES)
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    1e-5,
    decay_steps=50,
    decay_rate=0.9,
    staircase=True
)

model.compile(optimizer=keras.optimizers.Adam(lr_schedule), loss=keras.losses.BinaryCrossentropy(), metrics=['accuracy'])

Some layers from the model checkpoint at xlnet-base-cased were not used when initializing TFXLNetForSequenceClassification: ['lm_loss']
- This IS expected if you are initializing TFXLNetForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFXLNetForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFXLNetForSequenceClassification were not initialized from the model checkpoint at xlnet-base-cased and are newly initialized: ['sequence_summary', 'logits_proj']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [8]:
hist = model.fit(train_features, train_labels, validation_split=.1, batch_size=BATCH_SIZE, epochs=3, shuffle=True)

Epoch 1/3
209/209 [==============================] - 199s 880ms/step - loss: 0.6974 - accuracy: 0.5130 - val_loss: 0.6702 - val_accuracy: 0.6143
Epoch 2/3
 88/209 [===========>..................] - ETA: 1:42 - loss: 0.6967 - accuracy: 0.5311

KeyboardInterrupt: 

In [17]:
model.evaluate(dev_features, dev_labels, batch_size=BATCH_SIZE)

216/216 [==============================] - 43s 188ms/step - loss: 0.7716 - accuracy: 0.4645


[0.7716456055641174, 0.4645061790943146]

In [18]:
#model.save('model_checkpoints/dev62pct')

In [ ]:
out = model.predict(dev_features, batch_size=BATCH_SIZE)

In [ ]:
pos = np.where(out >= .5)
neg = np.where(out < .5)
print(f'Positives: {pos[0].shape[0]}')
print(f'Negatives: {neg[0].shape[0]}')